In [1]:
import numpy as np
import xarray as xr
import pandas as pd
from netCDF4 import Dataset
import os,datetime,sys,fnmatch
import time
import math
import dask.array as da
import graphviz

In [2]:
def read_MODIS_level2_data(MOD06_file,MOD03_file):
    print(MOD06_file)
    print(MOD03_file)
    print('reading the cloud mask from MOD06_L2 product')
    myd06 = Dataset(MOD06_file, "r")
    CM = myd06.variables["Cloud_Mask_1km"][:,:,:] # Reading Specific Variable 'Cloud_Mask_1km'.
    CM   = (np.array(CM[:,:,0],dtype='byte') & 0b00000110) >>1
    CM = np.array(CM).byteswap().newbyteorder()
    #CM = np.ravel(CM) # Changing The Shape Of The Variable 'Longitude'.
    print('level-2 cloud mask array shape',CM.shape)

    myd03 = Dataset(MOD03_file, "r")
    print('reading the lat-lon from MOD03 product')
    latitude = myd03.variables["Latitude"][:,:] # Reading Specific Variable 'Latitude'.
    #latitude = np.ravel(latitude) # Changing The Shape Of The Variable 'Latitude'.
    latitude = np.array(latitude).byteswap().newbyteorder() # Addressing Byteswap For Big Endian Error.
    longitude = myd03.variables["Longitude"][:,:] # Reading Specific Variable 'Longitude'.
    #longitude = np.ravel(longitude) # Changing The Shape Of The Variable 'Longitude'.
    longitude = np.array(longitude).byteswap().newbyteorder() # Addressing Byteswap For Big Endian Error.
    print('level-2 lat-lon array shape',latitude.shape)

    return latitude,longitude,CM

In [3]:
def value_locate(refx, x):
    refx = np.array(refx)
    x = np.array(x)
    loc = np.zeros(len(x), dtype='int')

    for i in range(len(x)):
        ix = x[i]
        ind = ((refx - ix) <= 0).nonzero()[0]
        if len(ind) == 0:
            loc[i] = -1
        else: loc[i] = ind[-1]

    return loc

def division(n, d):

    div = np.zeros(len(d))
    for i in range(len(d)):
        if d[i] >0:
          div[i]=n[i]/d[i]
        else: div[i]=None 

    return div

In [4]:
!set MOD03_PATH = /Users/saviosebastian/Documents/Project/CMAC/HDFFiles/

In [5]:
%env MOD03_PATH=/Users/saviosebastian/Documents/Project/CMAC/HDFFiles/
%env MOD06_PATH=/Users/saviosebastian/Documents/Project/CMAC/HDFFiles/

env: MOD03_PATH=/Users/saviosebastian/Documents/Project/CMAC/HDFFiles/
env: MOD06_PATH=/Users/saviosebastian/Documents/Project/CMAC/HDFFiles/


In [6]:
%env MOD03_PATH

'/Users/saviosebastian/Documents/Project/CMAC/HDFFiles/'

In [7]:
%env MOD06_PATH

'/Users/saviosebastian/Documents/Project/CMAC/HDFFiles/'

In [8]:
#userinput03 = input('Enter The File Location For MOD03 Files: ')
#MOD03_path = userinput03
#print(MOD03_path)

In [ ]:
MOD03_PATH

In [ ]:
userinput06 = input('Enter The File Location For MOD06 Files: ')
MOD06_path = userinput06
print(MOD06_path)

In [11]:
#MOD03_path = '/Users/saviosebastian/Documents/Project/CMAC/HDFFiles/'
#MOD06_path = '/Users/saviosebastian/Documents/Project/CMAC/HDFFiles/'
MOD03_path = %env MOD03_PATH
MOD06_path = %env MOD06_PATH
satellite = 'Aqua'

yr = [2008]
mn = [1] #np.arange(1,13)  #[1]
dy = [1] #np.arange(1,32) # [1] #np.arange(1,31)
# latitude and longtitude boundaries of level-3 grid
lat_bnd = np.arange(-90,91,1)
lon_bnd = np.arange(-180,180,1)
nlat = 180
nlon = 360

TOT_pix      = np.zeros(nlat*nlon)
CLD_pix      = np.zeros(nlat*nlon)

In [17]:
MOD03_fp = 'MYD03.A*.hdf'
MOD06_fp = 'MYD06_L2.A*.hdf'
MOD03_fn, MOD06_fn =[],[]
#MOD03_fn2, MOD06_fn2 =[],[]
MOD03_fn2, MOD06_fn2 =[],[]
for MOD06_flist in  os.listdir(MOD06_path):
    if fnmatch.fnmatch(MOD06_flist, MOD06_fp):
        MOD06_fn = MOD06_flist
        MOD06_fn2.append(MOD06_flist)
        print(MOD06_fn)
for MOD03_flist in  os.listdir(MOD03_path):
    if fnmatch.fnmatch(MOD03_flist, MOD03_fp):
        MOD03_fn = MOD03_flist
        MOD03_fn2.append(MOD03_flist)
        print(MOD03_fn)
if MOD03_fn and MOD06_fn:
    # if both MOD06 and MOD03 products are in the directory
    print('reading level 2 geolocation and cloud data')
    #print(MOD06_fn)
    #print(MOD03_fn)
    Lat,Lon,CM = read_MODIS_level2_data(MOD06_path+MOD06_fn,MOD03_path+MOD03_fn)
    

MYD06_L2.A2008001.0010.006.2013341192125.hdf
MYD06_L2.A2008001.0000.006.2013341193524.hdf
MYD06_L2.A2008001.0005.006.2013341193207.hdf
MYD03.A2008001.0010.006.2012066122416.hdf
MYD03.A2008001.0005.006.2012066122516.hdf
MYD03.A2008001.0000.006.2012066122450.hdf
reading level 2 geolocation and cloud data
/Users/saviosebastian/Documents/Project/CMAC/HDFFiles/MYD06_L2.A2008001.0005.006.2013341193207.hdf
/Users/saviosebastian/Documents/Project/CMAC/HDFFiles/MYD03.A2008001.0000.006.2012066122450.hdf
reading the cloud mask from MOD06_L2 product
level-2 cloud mask array shape (2030, 1354)
reading the lat-lon from MOD03 product
level-2 lat-lon array shape (2030, 1354)


In [18]:
MOD03_fn2

['MYD03.A2008001.0010.006.2012066122416.hdf',
 'MYD03.A2008001.0005.006.2012066122516.hdf',
 'MYD03.A2008001.0000.006.2012066122450.hdf']

In [19]:
MOD06_flist

'MYD03.A2008001.0000.006.2012066122450.hdf'

In [20]:
MOD06_fn2

['MYD06_L2.A2008001.0010.006.2013341192125.hdf',
 'MYD06_L2.A2008001.0000.006.2013341193524.hdf',
 'MYD06_L2.A2008001.0005.006.2013341193207.hdf']

In [21]:
MOD06_path

'/Users/saviosebastian/Documents/Project/CMAC/HDFFiles/'

In [22]:
%time
Lat

CPU times: user 4 µs, sys: 1 µs, total: 5 µs
Wall time: 11 µs


array([[69.99062 , 69.99425 , 69.99774 , ..., 62.61897 , 62.58984 ,
        62.560364],
       [69.97248 , 69.97615 , 69.9797  , ..., 62.605747, 62.57684 ,
        62.54706 ],
       [69.95433 , 69.95805 , 69.961655, ..., 62.592495, 62.563313,
        62.533596],
       ...,
       [52.11962 , 52.11983 , 52.119995, ..., 47.52033 , 47.50235 ,
        47.484215],
       [52.101448, 52.101715, 52.101936, ..., 47.503967, 47.48593 ,
        47.46775 ],
       [52.083275, 52.0836  , 52.083874, ..., 47.48759 , 47.469513,
        47.45127 ]], dtype=float32)

In [23]:
Lat.shape

(2030, 1354)

In [24]:
Lon.shape

(2030, 1354)

In [25]:
CM.shape

(2030, 1354)

In [26]:
#for items in MOD06_fn2:

In [34]:
myd06_name = '/Users/saviosebastian/Documents/Project/CMAC/HDFFiles/'
#cm = np.array(2)
cm = np.zeros((2030,1354), dtype=np.float32)
#cm = np.empty([2, 2])
#cm1 = np.array(2)
cm1 = np.zeros((2030,1354), dtype=np.float32)
#cm1 = np.empty([2, 2])
#cmr = np.array(2)
cmr = np.zeros((2030,1354), dtype=np.float32)
#cmr = np.empty([2, 2])
for MOD06_file in MOD06_fn2:
    MOD06_file2 = myd06_name + MOD06_file
    myd06 = Dataset(MOD06_file2, "r")
    CM = myd06.variables["Cloud_Mask_1km"][:,:,:]# Reading Specific Variable 'Cloud_Mask_1km'.
    CMr = myd06.variables["Cloud_Mask_1km"][:,:,0]
    CM1   = (np.array(CM[:,:,0],dtype='byte') & 0b00000001)
    CM   = (np.array(CM[:,:,0],dtype='byte') & 0b00000110) >>1
    CM = np.array(CM).byteswap().newbyteorder()
    cm = np.dstack((cm,CM))
    cm1 = np.dstack((cm1,CM1))
    cmr = np.dstack((cmr,CMr))
    #CM = np.ravel(CM) # Changing The Shape Of The Variable 'Longitude'.
    #ar.append(CM)
    #cm = np.append(cm, CM)
    #cm = np.concatenate((cm,CM), axis=0)
    #cm = np.column_stack([cm,CM])
    #cm1 = np.append(cm1, CM1)
    #cmr = np.append(cmr, CMr)
    #ar.insert(CM)
    print('level-2 cloud mask array shape',CM.shape)
    
    

level-2 cloud mask array shape (2030, 1354)
level-2 cloud mask array shape (2030, 1354)
level-2 cloud mask array shape (2030, 1354)


In [30]:
cmtemp = np.zeros((2030,1354), dtype=np.float32)
cmtemp

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [32]:
cm.shape

(2030, 1354, 4)

In [33]:
cm

array([[[0., 0., 0., 1.],
        [0., 0., 0., 1.],
        [0., 0., 0., 1.],
        ...,
        [0., 1., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]],

       [[0., 0., 0., 1.],
        [0., 0., 1., 1.],
        [0., 0., 0., 1.],
        ...,
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]],

       [[0., 0., 0., 1.],
        [0., 0., 1., 1.],
        [0., 0., 0., 2.],
        ...,
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]],

       ...,

       [[0., 3., 1., 0.],
        [0., 3., 1., 0.],
        [0., 3., 1., 0.],
        ...,
        [0., 3., 0., 0.],
        [0., 3., 0., 0.],
        [0., 3., 0., 0.]],

       [[0., 3., 1., 0.],
        [0., 3., 1., 2.],
        [0., 3., 0., 0.],
        ...,
        [0., 3., 0., 0.],
        [0., 3., 0., 0.],
        [0., 3., 0., 0.]],

       [[0., 3., 1., 1.],
        [0., 3., 1., 2.],
        [0., 3., 0., 2.],
        ...,
        [0., 1., 0., 0.],
        [0., 3.

In [37]:
cm1.shape

(2030, 1354, 4)

In [74]:
MOD06_fn2

['MYD06_L2.A2008001.0010.006.2013341192125.hdf',
 'MYD06_L2.A2008001.0000.006.2013341193524.hdf',
 'MYD06_L2.A2008001.0005.006.2013341193207.hdf']

In [71]:
cm1.shape

(8245864,)

In [88]:
cm

array([2, 0, 0, ..., 0, 0, 0])

In [39]:
cm = da.from_array(cm, chunks =(2030,1354,1))
cm1 = da.from_array(cm1, chunks =(2030,1354,1))
cmr = da.from_array(cmr, chunks =(2030,1354,1))

In [40]:
#cm = da.from_array(CM, chunks =(1015,677))

In [41]:
cm.compute()

array([[[0., 0., 0., 1.],
        [0., 0., 0., 1.],
        [0., 0., 0., 1.],
        ...,
        [0., 1., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]],

       [[0., 0., 0., 1.],
        [0., 0., 1., 1.],
        [0., 0., 0., 1.],
        ...,
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]],

       [[0., 0., 0., 1.],
        [0., 0., 1., 1.],
        [0., 0., 0., 2.],
        ...,
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]],

       ...,

       [[0., 3., 1., 0.],
        [0., 3., 1., 0.],
        [0., 3., 1., 0.],
        ...,
        [0., 3., 0., 0.],
        [0., 3., 0., 0.],
        [0., 3., 0., 0.]],

       [[0., 3., 1., 0.],
        [0., 3., 1., 2.],
        [0., 3., 0., 0.],
        ...,
        [0., 3., 0., 0.],
        [0., 3., 0., 0.],
        [0., 3., 0., 0.]],

       [[0., 3., 1., 1.],
        [0., 3., 1., 2.],
        [0., 3., 0., 2.],
        ...,
        [0., 1., 0., 0.],
        [0., 3.

In [43]:
cmtemp = da.from_array(cm, chunks =(500,500,1))
cmtemp.compute()

array([[[0., 0., 0., 1.],
        [0., 0., 0., 1.],
        [0., 0., 0., 1.],
        ...,
        [0., 1., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]],

       [[0., 0., 0., 1.],
        [0., 0., 1., 1.],
        [0., 0., 0., 1.],
        ...,
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]],

       [[0., 0., 0., 1.],
        [0., 0., 1., 1.],
        [0., 0., 0., 2.],
        ...,
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]],

       ...,

       [[0., 3., 1., 0.],
        [0., 3., 1., 0.],
        [0., 3., 1., 0.],
        ...,
        [0., 3., 0., 0.],
        [0., 3., 0., 0.],
        [0., 3., 0., 0.]],

       [[0., 3., 1., 0.],
        [0., 3., 1., 2.],
        [0., 3., 0., 0.],
        ...,
        [0., 3., 0., 0.],
        [0., 3., 0., 0.],
        [0., 3., 0., 0.]],

       [[0., 3., 1., 1.],
        [0., 3., 1., 2.],
        [0., 3., 0., 2.],
        ...,
        [0., 1., 0., 0.],
        [0., 3.

In [44]:
cmtemp.size

10994480

In [45]:
cmtemp.shape

(2030, 1354, 4)

In [47]:
cm.chunks

((2030,), (1354,), (1, 1, 1, 1))

In [52]:
myd03_name = '/Users/saviosebastian/Documents/Project/CMAC/HDFFiles/'
#lat = np.array(2)
lat = np.zeros((2030,1354), dtype=np.float32)
#lon = np.array(2)
lon = np.zeros((2030,1354), dtype=np.float32)
for MOD03_file in MOD03_fn2:
    MOD03_file2 = myd03_name + MOD03_file
    myd03 = Dataset(MOD03_file2, "r")
    latitude = myd03.variables["Latitude"][:,:] # Reading Specific Variable 'Latitude'.
    lat = np.dstack((lat,latitude))
    #latitude = np.ravel(latitude) # Changing The Shape Of The Variable 'Latitude'.
    #latitude = np.array(latitude).byteswap().newbyteorder() # Addressing Byteswap For Big Endian Error.
    #lat = np.append(lat, latitude)
    print('Latitude Shape Is: ',lat.shape)

    longitude = myd03.variables["Longitude"][:,:] # Reading Specific Variable 'Longitude'.
    #longitude = np.ravel(longitude) # Changing The Shape Of The Variable 'Longitude'.
    #longitude = np.array(longitude).byteswap().newbyteorder() # Addressing Byteswap For Big Endian Error.
    lon = np.dstack((lon,longitude))
    print('Latitude Shape Is: ',lon.shape)


    

Latitude Shape Is:  (2030, 1354, 2)
Latitude Shape Is:  (2030, 1354, 2)
Latitude Shape Is:  (2030, 1354, 3)
Latitude Shape Is:  (2030, 1354, 3)
Latitude Shape Is:  (2030, 1354, 4)
Latitude Shape Is:  (2030, 1354, 4)


In [53]:
lon.shape

(2030, 1354, 4)

In [54]:
lat = da.from_array(lat, chunks =(2030,1354,1))
lon = da.from_array(lon, chunks =(2030,1354,1))

In [56]:
cm.size

10994480

In [57]:
cm

dask.array<array, shape=(2030, 1354, 4), dtype=float32, chunksize=(2030, 1354, 1)>

In [58]:
lat.shape

(2030, 1354, 4)

In [99]:
MOD03_fn2

['MYD03.A2008001.0010.006.2012066122416.hdf',
 'MYD03.A2008001.0005.006.2012066122516.hdf',
 'MYD03.A2008001.0000.006.2012066122450.hdf']

In [59]:
lon.size

10994480

In [61]:
CMB123 = lat.astype(np.int8)
#CMB123 = xr.DataArray(CMB123)
CMB123 = da.from_array(CMB123, chunks =(2030,1354,1))
CMB124 = lon.astype(np.int8)
#CMB124 = xr.DataArray(CMB124)
CMB124 = da.from_array(CMB124, chunks =(2030,1354,1))

In [62]:
CMB123.compute()

array([[[ 0, 34, 52, 69],
        [ 0, 34, 52, 69],
        [ 0, 34, 52, 69],
        ...,
        [ 0, 30, 47, 62],
        [ 0, 30, 47, 62],
        [ 0, 30, 47, 62]],

       [[ 0, 34, 52, 69],
        [ 0, 34, 52, 69],
        [ 0, 34, 52, 69],
        ...,
        [ 0, 30, 47, 62],
        [ 0, 30, 47, 62],
        [ 0, 30, 47, 62]],

       [[ 0, 34, 52, 69],
        [ 0, 34, 52, 69],
        [ 0, 34, 52, 69],
        ...,
        [ 0, 30, 47, 62],
        [ 0, 30, 47, 62],
        [ 0, 30, 47, 62]],

       ...,

       [[ 0, 16, 34, 52],
        [ 0, 16, 34, 52],
        [ 0, 16, 34, 52],
        ...,
        [ 0, 13, 30, 47],
        [ 0, 13, 30, 47],
        [ 0, 13, 30, 47]],

       [[ 0, 16, 34, 52],
        [ 0, 16, 34, 52],
        [ 0, 16, 34, 52],
        ...,
        [ 0, 13, 30, 47],
        [ 0, 13, 30, 47],
        [ 0, 13, 30, 47]],

       [[ 0, 16, 34, 52],
        [ 0, 16, 34, 52],
        [ 0, 16, 34, 52],
        ...,
        [ 0, 13, 30, 47],
        [ 0, 13

In [64]:
#

dsa3 = xr.Dataset()
dsa3.coords['Latitude'] = (('x','y','z'), lat)
dsa3.coords['Longitude'] = (('x','y','z'), lon)
dsa3.coords['Bit0'] = (('x','y','z'), cm1)
dsa3.coords['Bit12'] = (('x','y','z'), cm)
dsa3.coords['LatInt'] = (('x','y','z'), CMB123)
dsa3.coords['LonInt'] = (('x','y','z'), CMB124)

In [66]:
dsa3

<xarray.Dataset>
Dimensions:    (x: 2030, y: 1354, z: 4)
Coordinates:
    Latitude   (x, y, z) float32 dask.array<shape=(2030, 1354, 4), chunksize=(2030, 1354, 1)>
    Longitude  (x, y, z) float32 dask.array<shape=(2030, 1354, 4), chunksize=(2030, 1354, 1)>
    Bit0       (x, y, z) float32 dask.array<shape=(2030, 1354, 4), chunksize=(2030, 1354, 1)>
    Bit12      (x, y, z) float32 dask.array<shape=(2030, 1354, 4), chunksize=(2030, 1354, 1)>
    LatInt     (x, y, z) int8 dask.array<shape=(2030, 1354, 4), chunksize=(2030, 1354, 1)>
    LonInt     (x, y, z) int8 dask.array<shape=(2030, 1354, 4), chunksize=(2030, 1354, 1)>
Dimensions without coordinates: x, y, z
Data variables:
    *empty*

In [67]:
cmr.size

10994480

In [68]:
CMrd = cmr.compute()
#CMrd = np.ravel(CMrd)

In [69]:
CMrd

array([[[   0.,   49.,   49.,  -13.],
        [   0.,   49.,   49.,  -13.],
        [   0.,   49.,   49.,  115.],
        ...,
        [   0.,  -77.,  -15., -111.],
        [   0.,  -79.,  -15., -111.],
        [   0.,  -79., -111., -111.]],

       [[   0.,   49.,   49.,  115.],
        [   0.,   49.,   51.,  115.],
        [   0.,   49.,   49.,  115.],
        ...,
        [   0.,  -79.,  -15., -111.],
        [   0.,  -79.,  -15., -111.],
        [   0.,  -79.,  -15., -111.]],

       [[   0.,   49.,   49.,  115.],
        [   0.,   49.,   51.,  115.],
        [   0.,   49.,   49.,  117.],
        ...,
        [   0.,  -79.,  -15., -111.],
        [   0.,  -79.,  -15., -111.],
        [   0.,  -79.,  -15., -111.]],

       ...,

       [[   0.,  -73.,  -13.,   49.],
        [   0.,  -73.,  -13.,   49.],
        [   0.,  -73.,  -13.,   49.],
        ...,
        [   0.,  119., -111.,  -79.],
        [   0.,  119., -111.,  -79.],
        [   0.,  -73., -111.,  -79.]],

       [[   0.,

In [70]:
dsa3['CloudFraction'] = (('x','y','z'), CMrd)

In [71]:
dsa3

<xarray.Dataset>
Dimensions:        (x: 2030, y: 1354, z: 4)
Coordinates:
    Latitude       (x, y, z) float32 dask.array<shape=(2030, 1354, 4), chunksize=(2030, 1354, 1)>
    Longitude      (x, y, z) float32 dask.array<shape=(2030, 1354, 4), chunksize=(2030, 1354, 1)>
    Bit0           (x, y, z) float32 dask.array<shape=(2030, 1354, 4), chunksize=(2030, 1354, 1)>
    Bit12          (x, y, z) float32 dask.array<shape=(2030, 1354, 4), chunksize=(2030, 1354, 1)>
    LatInt         (x, y, z) int8 dask.array<shape=(2030, 1354, 4), chunksize=(2030, 1354, 1)>
    LonInt         (x, y, z) int8 dask.array<shape=(2030, 1354, 4), chunksize=(2030, 1354, 1)>
Dimensions without coordinates: x, y, z
Data variables:
    CloudFraction  (x, y, z) float32 0.0 49.0 49.0 -13.0 ... 119.0 -111.0 -79.0

In [72]:
dsa3.Bit12.values

array([[[0., 0., 0., 1.],
        [0., 0., 0., 1.],
        [0., 0., 0., 1.],
        ...,
        [0., 1., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]],

       [[0., 0., 0., 1.],
        [0., 0., 1., 1.],
        [0., 0., 0., 1.],
        ...,
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]],

       [[0., 0., 0., 1.],
        [0., 0., 1., 1.],
        [0., 0., 0., 2.],
        ...,
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]],

       ...,

       [[0., 3., 1., 0.],
        [0., 3., 1., 0.],
        [0., 3., 1., 0.],
        ...,
        [0., 3., 0., 0.],
        [0., 3., 0., 0.],
        [0., 3., 0., 0.]],

       [[0., 3., 1., 0.],
        [0., 3., 1., 2.],
        [0., 3., 0., 0.],
        ...,
        [0., 3., 0., 0.],
        [0., 3., 0., 0.],
        [0., 3., 0., 0.]],

       [[0., 3., 1., 1.],
        [0., 3., 1., 2.],
        [0., 3., 0., 2.],
        ...,
        [0., 1., 0., 0.],
        [0., 3.

In [73]:
df5 = dsa3.to_dask_dataframe()

In [74]:
df5

,x,y,z,Latitude,Longitude,Bit0,Bit12,LatInt,LonInt,CloudFraction
npartitions=4,,,,,,,,,,
0,int64,int64,int64,float32,float32,float32,float32,int8,int8,float32
2745912,...,...,...,...,...,...,...,...,...,...
5491824,...,...,...,...,...,...,...,...,...,...
8237736,...,...,...,...,...,...,...,...,...,...
10994479,...,...,...,...,...,...,...,...,...,...


In [75]:
from dask.distributed import Client
client = Client()
client

Client Scheduler: tcp://127.0.0.1:57295 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 4 Cores: 4 Memory: 17.18 GB


In [76]:
%time
dsa4 = list(dsa3.groupby('LatInt', 'LonInt'))

CPU times: user 3 µs, sys: 2 µs, total: 5 µs
Wall time: 11 µs


distributed.utils_perf - WARNING - full garbage collections took 24% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 38% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 38% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 39% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 39% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 40% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 40% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 40% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 41% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took

Traceback (most recent call last):
  File "/Users/saviosebastian/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 3267, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-76-73ef223fedea>", line 2, in <module>
    dsa4 = list(dsa3.groupby('LatInt', 'LonInt'))
  File "/Users/saviosebastian/anaconda3/lib/python3.6/site-packages/xarray/core/groupby.py", line 290, in _iter_grouped
    yield self._obj.isel(**{self._group_dim: indices})
  File "/Users/saviosebastian/anaconda3/lib/python3.6/site-packages/xarray/core/dataset.py", line 1509, in isel
    new_var = var.isel(indexers=var_indexers)
  File "/Users/saviosebastian/anaconda3/lib/python3.6/site-packages/xarray/core/variable.py", line 908, in isel
    return self[tuple(key)]
  File "/Users/saviosebastian/anaconda3/lib/python3.6/site-packages/xarray/core/variable.py", line 618, in __getitem__
    data = as_indexable(self._data)[indexer]
  File "/Users/saviosebastian/anacon

KeyboardInterrupt: 

distributed.nanny - WARNING - Worker process 3067 was killed by unknown signal
distributed.nanny - WARNING - Worker process 3068 was killed by unknown signal
distributed.nanny - WARNING - Worker process 3070 was killed by unknown signal
distributed.nanny - WARNING - Worker process 3071 was killed by unknown signal
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Worker exceeded 95% memory budget. Restarting
distributed.nanny - WARNING - Worker process 3075 was killed by unknown signal
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Worker process 3073 was killed by unknown signal
distributed.nanny - WARNING - Worker process 3074 was killed by unknown signal
distributed.nanny - WARNING - Worker process 3072 was killed by unknown signal
distributed.nanny - WARNING - Worker process 3076 wa

In [114]:
dsa4.index

<function list.index>

In [71]:
%time
dsa4

CPU times: user 2 µs, sys: 1 µs, total: 3 µs
Wall time: 7.87 µs


[(2, <xarray.Dataset>
  Dimensions:        (x: 1)
  Coordinates:
      Latitude       (x) float64 dask.array<shape=(1,), chunksize=(1,)>
      Longitude      (x) float64 dask.array<shape=(1,), chunksize=(1,)>
      Bit0           (x) int64 dask.array<shape=(1,), chunksize=(1,)>
      Bit12          (x) int64 dask.array<shape=(1,), chunksize=(1,)>
      LatInt         (x) int8 dask.array<shape=(1,), chunksize=(1,)>
      LonInt         (x) int8 dask.array<shape=(1,), chunksize=(1,)>
  Dimensions without coordinates: x
  Data variables:
      CloudFraction  (x) int64 2), (13, <xarray.Dataset>
  Dimensions:        (x: 6702)
  Coordinates:
      Latitude       (x) float64 dask.array<shape=(6702,), chunksize=(6702,)>
      Longitude      (x) float64 dask.array<shape=(6702,), chunksize=(6702,)>
      Bit0           (x) int64 dask.array<shape=(6702,), chunksize=(6702,)>
      Bit12          (x) int64 dask.array<shape=(6702,), chunksize=(6702,)>
      LatInt         (x) int8 dask.array<shape=(

In [115]:
list1=[]
for key, value in dsa4:
    #print(key)
    list1.append(key)

In [116]:
Nobs = np.sum(cm1.compute()>=0)
print("Number Of Total Pixel: ", np.sum(cm1.compute()>=0), " Pixels")
print("Number Of Pixel With Cloud Mask Determined: ", np.sum(cm1.compute()==1), " Pixels")

Number Of Total Pixel:  8245861  Pixels
Number Of Pixel With Cloud Mask Determined:  8245860  Pixels


In [117]:
Nobs = np.sum(cm1>=0)
print("Number Of Total Pixel: ", np.sum(cm1>=0), " Pixels")
print("Number Of Pixel With Cloud Mask Determined: ", np.sum(cm1==1), " Pixels")

Number Of Total Pixel:  dask.array<sum-aggregate, shape=(), dtype=int64, chunksize=()>  Pixels
Number Of Pixel With Cloud Mask Determined:  dask.array<sum-aggregate, shape=(), dtype=int64, chunksize=()>  Pixels


In [ ]:
%time
i = 0
while i < len(list1):
    #print(list1[i])
    a = list1[i]
    #print(dsa4[i])
    dsa4 =  list(dsa3.groupby('LatInt', 'LonInt'))
    dsa41 = dsa4[0][1]
    dsa51 = list(dsa41.groupby('LonInt'))
    dsa61 = dsa51[0][1] 
    dsa71 = list(dsa61.groupby('Bit0', 'Bit12'))
    dsa81 = dsa71[0][1]
    dsa91 = list(dsa81.groupby('Bit12'))
    dsa81 = dsa71[0][1]
    dsa91 = dsa81.groupby('Bit12').count()/Nobs*100
    print(dsa91)
    i += 1

In [120]:
%time
i = 0
start_time = time.time()
while i < len(list1):
    #print(list1[i])
    a = list1[i]
    #print(dsa4[i])
    dsa4 =  list(dsa3.groupby('LatInt', 'LonInt'))
    dsa41 = dsa4[i][1]
    dsa51 = list(dsa41.groupby('LonInt'))
    dsa61 = dsa51[0][1] 
    dsa71 = list(dsa61.groupby('Bit0', 'Bit12'))
    dsa81 = dsa71[0][1]
    dsa91 = list(dsa81.groupby('Bit12'))
    dsa81 = dsa71[0][1]
    dsa91 = dsa81.groupby('Bit12').count()
    print(dsa91)
    i += 1
end_time = time.time()
print("Total Time Taken This Loop: ", end_time - start_time)
hours, rem = divmod(end_time-start_time, 3600)
minutes, seconds = divmod(rem, 60)
print("{:0>2}:{:0>2}:{:05.2f}".format(int(hours),int(minutes),seconds))
%time

CPU times: user 3 µs, sys: 3 µs, total: 6 µs
Wall time: 15 µs


/Users/saviosebastian/anaconda3/lib/python3.6/site-packages/xarray/core/groupby.py:723: FutureWarning: Default reduction dimension will be changed to the grouped dimension after xarray 0.12. To silence this warning, pass dim=xarray.ALL_DIMS explicitly.
  **kwargs)


<xarray.Dataset>
Dimensions:        (Bit12: 1)
Coordinates:
  * Bit12          (Bit12) int64 2
Data variables:
    CloudFraction  (Bit12) int64 1
<xarray.Dataset>
Dimensions:        (Bit12: 1)
Coordinates:
  * Bit12          (Bit12) int64 3
Data variables:
    CloudFraction  (Bit12) int64 372
<xarray.Dataset>
Dimensions:        (Bit12: 2)
Coordinates:
  * Bit12          (Bit12) int64 2 3
Data variables:
    CloudFraction  (Bit12) int64 1 24
<xarray.Dataset>
Dimensions:        (Bit12: 1)
Coordinates:
  * Bit12          (Bit12) int64 3
Data variables:
    CloudFraction  (Bit12) int64 12
<xarray.Dataset>
Dimensions:        (Bit12: 4)
Coordinates:
  * Bit12          (Bit12) int64 0 1 2 3
Data variables:
    CloudFraction  (Bit12) int64 2 46 49 1121
<xarray.Dataset>
Dimensions:        (Bit12: 3)
Coordinates:
  * Bit12          (Bit12) int64 1 2 3
Data variables:
    CloudFraction  (Bit12) int64 2 36 1150
<xarray.Dataset>
Dimensions:        (Bit12: 4)
Coordinates:
  * Bit12          (Bit12) 

In [ ]:
%time
i = 0
while i < len(list1):
    #print(list1[i])
    a = list1[i]
    #print(dsa4[i])
    dsa4 =  list(dsa3.groupby('LatInt', 'LonInt'))
    dsa41 = dsa4[i][1]
    dsa51 = list(dsa41.groupby('LonInt'))
    dsa61 = dsa51[0][1] 
    dsa71 = list(dsa61.groupby('Bit0', 'Bit12'))
    dsa81 = dsa71[0][1]
    dsa91 = list(dsa81.groupby('Bit12'))
    dsa81 = dsa71[0][1]
    dsa91 = dsa81.groupby('Bit12')
    print(dsa91)
    i += 1

In [198]:
time.time()

1546942482.3784

In [117]:
%time
start_time = time.time()
i = 0

while i < len(list1):
    #print(list1[i])
    a = list1[i]
    #print(dsa4[i])
    dsa4 =  list(dsa3.groupby('LatInt', 'LonInt'))
    dsa41 = dsa4[i][1]
    dsa51 = list(dsa41.groupby('LonInt'))
    dsa61 = dsa51[0][1] 
    dsa71 = list(dsa61.groupby('Bit0', 'Bit12'))
    dsa81 = dsa71[0][1]
    dsa91 = list(dsa81.groupby('Bit12'))
    dsa81 = dsa71[0][1]
    dsa91 = list(dsa81.groupby('Bit12'))
    print(dsa91)
    i += 1

end_time = time.time()
print("Total Time Taken This Loop: ", end_time - start_time)
hours, rem = divmod(end_time-start_time, 3600)
minutes, seconds = divmod(rem, 60)
print("{:0>2}:{:0>2}:{:05.2f}".format(int(hours),int(minutes),seconds))

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 5.96 µs
[(2, <xarray.Dataset>
Dimensions:        (x: 1)
Coordinates:
    Latitude       (x) float64 2.0
    Longitude      (x) float64 2.0
    Bit0           (x) int64 2
    Bit12          (x) int64 dask.array<shape=(1,), chunksize=(1,)>
    LatInt         (x) int8 2
    LonInt         (x) int8 2
Dimensions without coordinates: x
Data variables:
    CloudFraction  (x) int64 2)]
[(3, <xarray.Dataset>
Dimensions:        (x: 372)
Coordinates:
    Latitude       (x) float64 14.0 13.99 13.99 14.0 ... 13.84 13.84 13.83 13.83
    Longitude      (x) float64 28.95 28.97 28.99 28.88 ... 28.94 28.96 28.98
    Bit0           (x) int64 1 1 1 1 1 1 1 1 1 1 1 1 ... 1 1 1 1 1 1 1 1 1 1 1 1
    Bit12          (x) int64 dask.array<shape=(372,), chunksize=(372,)>
    LatInt         (x) int8 13 13 13 13 13 13 13 13 ... 13 13 13 13 13 13 13 13
    LonInt         (x) int8 28 28 28 28 28 28 28 28 ... 28 28 28 28 28 28 28 28
Dimensions without coordinate

In [118]:
from dask.distributed import Client
client = Client()

In [119]:
client.cluster

In [122]:
from dask.distributed import Client

# Setup a local cluster.
# By default this sets up 1 worker per core
client = Client()
client

Client Scheduler: tcp://127.0.0.1:54707,Cluster Workers: 4 Cores: 4 Memory: 17.18 GB


In [123]:
%time
start_time = time.time()
i = 0

while i < len(list1):
    #print(list1[i])
    a = list1[i]
    #print(dsa4[i])
    dsa4 =  list(dsa3.groupby('LatInt', 'LonInt'))
    dsa41 = dsa4[i][1]
    dsa51 = list(dsa41.groupby('LonInt'))
    dsa61 = dsa51[0][1] 
    dsa71 = list(dsa61.groupby('Bit0', 'Bit12'))
    dsa81 = dsa71[0][1]
    dsa91 = list(dsa81.groupby('Bit12'))
    dsa81 = dsa71[0][1]
    dsa91 = list(dsa81.groupby('Bit12'))
    print(dsa91)
    i += 1

end_time = time.time()
print("Total Time Taken This Loop: ", end_time - start_time)
hours, rem = divmod(end_time-start_time, 3600)
minutes, seconds = divmod(rem, 60)
print("{:0>2}:{:0>2}:{:05.2f}".format(int(hours),int(minutes),seconds))

CPU times: user 3 µs, sys: 1e+03 ns, total: 4 µs
Wall time: 11 µs
[(2, <xarray.Dataset>
Dimensions:        (x: 1)
Coordinates:
    Latitude       (x) float64 2.0
    Longitude      (x) float64 2.0
    Bit0           (x) int64 2
    Bit12          (x) int64 dask.array<shape=(1,), chunksize=(1,)>
    LatInt         (x) int8 2
    LonInt         (x) int8 2
Dimensions without coordinates: x
Data variables:
    CloudFraction  (x) int64 2)]
[(3, <xarray.Dataset>
Dimensions:        (x: 372)
Coordinates:
    Latitude       (x) float64 14.0 13.99 13.99 14.0 ... 13.84 13.84 13.83 13.83
    Longitude      (x) float64 28.95 28.97 28.99 28.88 ... 28.94 28.96 28.98
    Bit0           (x) int64 1 1 1 1 1 1 1 1 1 1 1 1 ... 1 1 1 1 1 1 1 1 1 1 1 1
    Bit12          (x) int64 dask.array<shape=(372,), chunksize=(372,)>
    LatInt         (x) int8 13 13 13 13 13 13 13 13 ... 13 13 13 13 13 13 13 13
    LonInt         (x) int8 28 28 28 28 28 28 28 28 ... 28 28 28 28 28 28 28 28
Dimensions without coordina

In [30]:
from dask.distributed import Client
client = Client()
client

Client Scheduler: tcp://127.0.0.1:54904 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 4 Cores: 4 Memory: 17.18 GB
